In [1]:
import os
import warnings
import time
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
warnings.filterwarnings("ignore")
DATA_DIR = "/home/bookboi/Nextcloud/1_Docs/4_Master/20_Studienarbeit_paper/data/datensatz_dominik/2 (copy)/"
data_file = "01_WorldPositions_zweiteAuslieferung.csv"
os.chdir("/home/bookboi/Nextcloud/1_Docs/4_Master/21_Studienarbeit_coding/")
from calibration_tool.data_integration.data_set import DataSet
from calibration_tool.data_integration.case_identification import (
    following_cars, car_goes_straight)
from calibration_tool.data_integration.helper import get_lane, get_angle
data_set = DataSet(DATA_DIR)
data_frame, meta_data, lane_data = data_set.get_dataframes_by_id("01")
use_xy = False
crossing_points = [
    9.191343,
    48.788692,
    9.191638,
    48.788567
]
dtypes = {
    "leader": int,
    "follower": float,
    "lane": int,
    "leaderIntersectionCrossing": float,
    "followerIntersectionCrossing": float,
    "recordingId": int
}

In [ ]:
pairs = pd.read_csv(str(Path(DATA_DIR) / "01_WorldPositions_selection.csv"),index_col=0,
                    dtype=dtypes)
inspect_objects = pairs.values[:,:2]
# np.random.shuffle(inspect_objects)
inspect_objects = [(99, np.nan), (209, np.nan),  (204, np.nan), (211, np.nan), (215, np.nan), (115, np.nan)]
abscissa = "xCenter" if use_xy else "lon"
ordinate = "yCenter" if use_xy else "lat"
for leader, follower in inspect_objects[:1]:
    leader = int(leader)
    follower = "N/A" if np.isnan(follower) else int(follower)
    times = data_frame[data_frame["trackId"]==leader]["time"].values
    rec_id = meta_data["recordingId"].values[0]
    actual_time = \
        data_frame[data_frame["trackId"]==leader]["intersectionCrossing"]
    if actual_time.values.shape[0] == 0:
        continue
    rec_id = meta_data["recordingId"].values[0]
    actual_time = actual_time.values[0]
    actual_time = times[np.argmin(np.abs(times-actual_time))]
    # actual_time=6.2
    actual_frame = (
        data_frame[(data_frame["trackId"]==leader)
                    & (data_frame["time"]==actual_time)]["frame"].values[0]
    ).astype(int)
    fig, ax = plt.subplots(figsize=(14,14))
    ax.set_title(f"{rec_id}@{time.strftime('%H:%M:%S', time.gmtime(actual_time))} L={leader:d} F={follower}")
    ax.plot(crossing_points[::2], crossing_points[1::2], linestyle="dashed")
    for lane_id in lane_data["trackId"].unique():
        lane_chunk = lane_data[lane_data["trackId"]==lane_id]
        lon, lat = lane_chunk[abscissa].values, lane_chunk[ordinate].values
        ax.plot(lon, lat, linewidth=0.5)
    cars_at_time = [2]
    for track_id in data_frame["trackId"].unique():
        data_chunk = data_frame[data_frame["trackId"]==track_id]
        conditions = (
            any(data_chunk["time"].values == actual_time))
        if conditions:
            cars_at_time.append(track_id)
    for track_id in cars_at_time:
        data_chunk = data_frame[data_frame["trackId"]==track_id]
        # if not car_goes_straight(lane_data, data_chunk):
        #     continue
        lon = data_chunk[data_chunk["time"]==actual_time][abscissa].values
        lat = data_chunk[data_chunk["time"]==actual_time][ordinate].values
        next_frame = actual_frame + 1 
        dlon = data_chunk[data_chunk["frame"]==next_frame][abscissa].values - lon
        dlat = data_chunk[data_chunk["frame"]==next_frame][ordinate].values - lat
        if dlon.shape[0] == 0:
            continue
        ax.scatter(lon, lat)
        if data_chunk["class"].values[0] != "Car":
            class_string = data_chunk["class"].values[0]
            ax.annotate(f"{track_id}|{class_string}", [lon, lat], fontsize=10)
        else:
            ax.annotate(f"{track_id}", [lon, lat], fontsize=10)
        colormap = {leader: 'r', follower: 'g'}
        color = colormap.get(track_id) or 'b'
        ax.arrow(lon[0], lat[0], dlon[0], dlat[0], width=0.0000125, color=color)
    current_values = ax.get_yticks()
    ax.set_yticklabels(['{:.3f}'.format(x) for x in current_values])
    current_values = ax.get_xticks()
    ax.set_xticklabels(['{:.3f}'.format(x) for x in current_values])
    plt.show()

/tmp/ipykernel_327793/486888831.py:60: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels(['{:.3f}'.format(x) for x in current_values])


In [ ]:
fig, ax = plt.subplots()
crossing_points = [
        9.191343,
        48.788692,
        9.191638,
        48.788567
    ]
for lane_id in lane_data["trackId"].unique():
        lane_chunk = lane_data[lane_data["trackId"]==lane_id]
        lon, lat = lane_chunk["lon"].values, lane_chunk["lat"].values
        ax.plot(lon, lat, linewidth=0.5)
data_chunk = data_frame[data_frame["trackId"]==718]
ax.plot(crossing_points[::2], crossing_points[1::2], linestyle="dashed")
data_chunk.plot(x="lon", y="lat", ax=ax)

plt.show()

In [ ]:
for data_frame, meta_data, lane_data in data_set.get_next_dataframes():
    for leader, follower in following_cars(data_frame, lane_data):
        print(f"leader={leader} follower={follower}")
    fig, ax = plt.subplots()
    data_chunk = data_frame[data_frame["trackId"]==467]
    get_lane(data_chunk["lon"].values, data_chunk["lat"].values, lane_data)
    car_goes_straight(lane_data, data_chunk)
    for lane_id in lane_data["trackId"].unique():
        lane_chunk = lane_data[lane_data["trackId"]==lane_id]
        lon, lat = lane_chunk["lon"].values, lane_chunk["lat"].values
        ax.plot(lon, lat, linewidth=0.5)
    ax.plot(data_chunk["lon"].values, data_chunk["lat"].values)
    plt.show()
    break